# Memory / Chat

Wir haben schon das Chat-Modell von OpenAI genutzt. Allerdings haben wir das noch nicht wirklich als Chat benutzt, da die bisherigen Aufgaben stets nur ein Aufgaben + Antwortpaar produziert haben.
Wichtig zu wissen ist, dass die ganzen Sprachmodelle, die wir für Chat-Bots nutzen, keine Daten sich behalten. D.h. der Chatverlauf existiert dort nicht. Wenn man also eine Frage stellt, die sich auf vorherige Daten aus dem Verlauf beziehen, dann weiß die KI nichts mehr darüber.
Damit das Ganze trotzdem möglich ist, muss man stets die gesamte (oder Teile) der Konversation bei jedem neuen Prompt mitschicken. Der Chatverlauf wird dadurch zum Teil des Kontext und die KI hat ein Kurzzeitgedächtnis bekommen.

Erstmal ein Beispiel, wie man das zu Fuß macht. Danach schauen wir uns an, wie man das mit LangChain automatisieren kann.

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="Du bist ein hilfreicher KI Assistent, der mir bei meinen Fragen und Aufgaben hilft"),
    ]

llm = ChatOpenAI()

user_prompt = "Erstelle bitte eine Liste der fünf größten Sprachmodelle" # Hier könnten wir auch tatsächlich eine Eingabeaufforderung starten. Zum Zeigen gebe ich aber den Text vor
messages.append(HumanMessage(content=user_prompt))

response = llm(messages)
messages.append(response)
print(response)

messages.append(HumanMessage(content="Welche stammen von OpenAI?"))

response = llm(messages)
messages.append(response)
print(response)

content='Hier ist eine Liste der fünf größten Sprachmodelle:\n\n1. GPT-3 (Generative Pre-trained Transformer 3) von OpenAI: GPT-3 ist eines der größten Sprachmodelle, das derzeit existiert. Es besteht aus 175 Milliarden Parametern und kann komplexe Aufgaben wie Übersetzung, Textgenerierung und Frage-Antwort-Interaktionen bewältigen.\n\n2. T5 (Text-to-Text Transfer Transformer) von Google: T5 ist ein leistungsstarkes Sprachmodell mit 11 Milliarden Parametern. Es kann verschiedene sprachliche Aufgaben wie Übersetzung, Textzusammenfassung, Textklassifikation und vieles mehr bewältigen.\n\n3. Megatron-LM von NVIDIA: Megatron-LM ist ein weiteres beeindruckendes Sprachmodell mit 8,3 Milliarden Parametern. Es wurde speziell für die Verarbeitung großer Mengen von Textdaten entwickelt und kann Aufgaben wie Textgenerierung und Sprachverständnis bewältigen.\n\n4. T-NLG (Text-Natural Language Generation) von Microsoft: T-NLG ist ein Sprachmodell mit beeindruckenden 17 Milliarden Parametern. Es wur

Erstmal sehen wir, dass wir unterschiedliche Message-Typen haben: SystemMessage, Humanmessage und AIMessage. Diese widerspiegeln die verschiedenen Rollen, die von ChatGPT erwartet werden.
Der Ablauf ist stets so, dass wir der Liste der Nachrichten die Fragen und Antworten hinzufügen. Die _messages_ Liste ist somit unser Chatverlauf und praktisch unser Speicher.

Damit wir das nicht manuell machen müssen, bietet LangChain _Memory_ an. Davon gibt es unterschiedliche Implementierungen. Der einfachste Speicher merkt sich ganz einfach jede Nachricht, so wie wir es oben gemacht haben. Allerdings ist der Platz im Kontext begrenzt und ein Chat, der länger geführt wird, könnte dazu führen, dass die maximale Kontextgröße überschritten wird. Dafür gibt es Speicher, die entweder nur die letzten n Nachrichten behält oder noch ausgefeilter, ein Speicher, der die bisherige Kommunikation zusammenfasst, um Platz zu sparen.

Schauen wir uns das Beispiel von oben mit LangChain Memory an:

In [8]:
from langchain.memory import ChatMessageHistory

llm = ChatOpenAI()

history = ChatMessageHistory()

history.add_message(SystemMessage(content="Du bist ein hilfreicher KI Assistent, der mir bei meinen Fragen und Aufgaben hilft"))

history.add_user_message("Erstelle bitte eine Liste der fünf größten Sprachmodelle")

response = llm(history.messages)

history.add_ai_message(response.content)

print(response)

content='1. GPT-3 (Generative Pre-trained Transformer 3): GPT-3 ist eines der größten Sprachmodelle, das von OpenAI entwickelt wurde. Es besteht aus 175 Milliarden Parametern und ist bekannt für seine Fähigkeit, menschenähnlichen Text zu generieren.\n\n2. Turing NLG (Natural Language Generation): Turing NLG ist ein Sprachmodell, das von Microsoft entwickelt wurde. Es besteht aus 17 Milliarden Parametern und wurde speziell für die Generierung von natürlicher Sprache entwickelt.\n\n3. T5 (Text-to-Text Transfer Transformer): T5 ist ein Sprachmodell, das von Google entwickelt wurde. Es besteht aus 11 Milliarden Parametern und kann für eine Vielzahl von Aufgaben verwendet werden, darunter Textklassifikation, Übersetzung, Zusammenfassung und vieles mehr.\n\n4. Megatron-LM: Megatron-LM ist ein Sprachmodell, das von NVIDIA entwickelt wurde. Es besteht aus bis zu 8,3 Milliarden Parametern und ist für seine Fähigkeit bekannt, große Mengen an Textdaten zu verarbeiten und zu analysieren.\n\n5. Pro

Das Beispiel soll zeigen, wie die Grundstrukturen eines Speichers aussehen. Im Grunde genommen ist das mit den manuellen Schritten erstmal fast identisch und hilft uns erstmal nicht das Ganze zu automatisieren.
In Kombination mit einer _vernünftigen_ Speicher-Implementierung und einer Chain sieht das Ganze wesentlich besser aus:

In [9]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()

llm = ChatOpenAI()

conversation = ConversationChain(
    llm=llm, 
    memory=memory
)

response = conversation.predict(input="Welche Bedeutung hat die Angabe der Anzahl der Parameter bei einem Sprachmodell?")

print(response)

Die Angabe der Anzahl der Parameter bei einem Sprachmodell gibt an, wie viele verschiedene Gewichte und Verbindungen das Modell hat. Je mehr Parameter ein Sprachmodell hat, desto komplexer und leistungsfähiger kann es sein. Eine höhere Anzahl von Parametern ermöglicht es dem Modell, eine größere Vielfalt an sprachlichen Nuancen zu erfassen und komplexe Zusammenhänge besser zu verstehen. Allerdings erhöht eine höhere Anzahl von Parametern auch die Rechen- und Speicheranforderungen des Modells. Es ist wichtig, ein Gleichgewicht zwischen der Modellgröße und der Leistungsfähigkeit zu finden, um gute Ergebnisse zu erzielen.


In [10]:
response = conversation.predict(input="Wie viele sind es bei GPT-3?")

print(response)

GPT-3 hat insgesamt 175 Milliarden Parameter. Das macht es zu einem der größten und leistungsfähigsten Sprachmodelle, die derzeit verfügbar sind. Aufgrund dieser enormen Anzahl an Parametern kann GPT-3 ein breites Spektrum an Aufgaben bewältigen und komplexe Zusammenhänge in der Sprache verstehen. Es ist jedoch wichtig zu beachten, dass die große Anzahl an Parametern auch hohe Rechen- und Speicherressourcen erfordert.


Wir sehen also, dass die Chain sich automatisch darum kümmert die Nachrichten in den dazugehörigen Speicher zu laden. Durch die Verwendung von _ConversationChain_ haben wir automatisch ein Prompttemplate, das einen _input_ Parameter erwartet, den wir an die _predict_ Funktion übergeben.

In [11]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Welche Bedeutung hat die Angabe der Anzahl der Parameter bei einem Sprachmodell?'), AIMessage(content='Die Angabe der Anzahl der Parameter bei einem Sprachmodell gibt an, wie viele verschiedene Gewichte und Verbindungen das Modell hat. Je mehr Parameter ein Sprachmodell hat, desto komplexer und leistungsfähiger kann es sein. Eine höhere Anzahl von Parametern ermöglicht es dem Modell, eine größere Vielfalt an sprachlichen Nuancen zu erfassen und komplexe Zusammenhänge besser zu verstehen. Allerdings erhöht eine höhere Anzahl von Parametern auch die Rechen- und Speicheranforderungen des Modells. Es ist wichtig, ein Gleichgewicht zwischen der Modellgröße und der Leistungsfähigkeit zu finden, um gute Ergebnisse zu erzielen.'), HumanMessage(content='Wie viele sind es bei GPT-3?'), AIMessage(content='GPT-3 hat insgesamt 175 Milliarden Parameter. Das macht es zu einem der größten und leistungsfähigsten Spr